In [3]:
import random
import operator
from functools import partial

!pip install deap # Install the deap library
from deap import base, creator, tools, gp

# Para simplificar expresiones booleanas
try:
    import sympy as sp
    HAS_SYMPY = True
except Exception:
    HAS_SYMPY = False

# -------------  Mapa estándar BCD -> segmentos a..g -------------
# Tabla de referencia: cada entrada es una tupla (a,b,c,d,e,f,g) con 1=segmento encendido.
# Fuente: estándar de 7-segment displays (ej. electronicsforu). :contentReference[oaicite:5]{index=5}
SEVEN_SEG = {
    0: (1,1,1,1,1,1,0),
    1: (0,1,1,0,0,0,0),
    2: (1,1,0,1,1,0,1),
    3: (1,1,1,1,0,0,1),
    4: (0,1,1,0,0,1,1),
    5: (1,0,1,1,0,1,1),
    6: (1,0,1,1,1,1,1),
    7: (1,1,1,0,0,0,0),
    8: (1,1,1,1,1,1,1),
    9: (1,1,1,1,0,1,1),
}

# Inputs will be x3 x2 x1 x0 (x3 MSB). We'll map input integer -> bit tuple.
def int_to_bits4(n):
    return ((n >> 3) & 1, (n >> 2) & 1, (n >> 1) & 1, n & 1)

# -------------  Operadores lógicos seguros (retornan 0/1) -------------
def AND(a, b):
    return int(bool(a) and bool(b))

def OR(a, b):
    return int(bool(a) or bool(b))

def XOR(a, b):
    return int(bool(a) ^ bool(b))

def NOT(a):
    return int(not bool(a))

# Optional helpers for readable printing with sympy later
def expr_to_sympy(expr_str, input_names):
    """
    Convierte una expresión string estilo DEAP (p.ej. OR(AND(x0,NOT(x1)),x2))
    en una expresión sympy booleana (si sympy está disponible).
    """
    if not HAS_SYMPY:
        return None
    # Map DEAP names to sympy equivalents
    local_dict = {
        'AND': lambda a,b: sp.And(a,b),
        'OR': lambda a,b: sp.Or(a,b),
        'XOR': lambda a,b: sp.Xor(a,b),
        'NOT': lambda a: sp.Not(a),
    }
    # Símbolos
    syms = {name: sp.symbols(name) for name in input_names}
    local_dict.update(syms)
    # Use sympy.sympify with a mapping
    try:
        sym = sp.sympify(expr_str, locals=local_dict)
        return sym
    except Exception:
        return None

# -------------  Construcción del problema GP con DEAP -------------
def make_pset():
    # 4 entradas: x0..x3
    pset = gp.PrimitiveSet("MAIN", 4)
    # La convención de nombres: argumentos serán x0, x1, x2, x3
    # por defecto DEAP las nombra ARG0..ARGn; cambiamos los nombres
    pset.renameArguments(ARG0='x3')  # MSB
    pset.renameArguments(ARG1='x2')
    pset.renameArguments(ARG2='x1')
    pset.renameArguments(ARG3='x0')  # LSB

    # Añadir primitivas lógicas
    pset.addPrimitive(AND, 2)
    pset.addPrimitive(OR, 2)
    pset.addPrimitive(XOR, 2)
    pset.addPrimitive(NOT, 1)

    # Constantes lógicas (0/1)
    pset.addTerminal(0)
    pset.addTerminal(1)

    return pset

def setup_deap(pset, parsimony_coefficient=0.01):
    """
    Crea los objetos creator y toolbox de DEAP para GP.
    """
    creator_name = "FitnessMax"
    try:
        creator.FitnessMax
    except Exception:
        creator.create("FitnessMax", base.Fitness, weights=(1.0,))

    try:
        creator.Individual
    except Exception:
        creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

    toolbox = base.Toolbox()

    # Generadores de expresiones iniciales
    toolbox.register("expr_init", gp.genFull, pset=pset, min_=1, max_=3)
    toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr_init)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    # Operators
    toolbox.register("compile", gp.compile, pset=pset)

    # Genetic operators (crossover / mutation)
    toolbox.register("select", tools.selTournament, tournsize=3)
    toolbox.register("mate", gp.cxOnePoint)
    # mutUniform replaces a subtree
    toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
    toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

    # Decorators to limit tree height
    toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
    toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

    # We'll create an evaluation function on the fly (per-segment), since the target segment changes.
    return toolbox

def evaluate_individual(expr_func, target_values, samples, individual, parsimony_coefficient):
    """
    expr_func: callable compiled from tree
    target_values: list of expected outputs for each sample (0/1)
    samples: list of input bit-tuples corresponding to target_values
    individual: tree (used to compute size)
    """
    correct = 0
    for bits, target in zip(samples, target_values):
        try:
            out = expr_func(*bits)
            outb = int(bool(out))
        except Exception:
            # runtime error in expression -> worst
            outb = 0
        if outb == target:
            correct += 1
    # Parsimony penalty
    size = len(individual)
    fitness = correct - parsimony_coefficient * size
    # DEAP expects a tuple
    return (fitness,)

def run_gp_for_segment(segment_idx,
                       pset,
                       toolbox,
                       population_size=300,
                       ngen=40,
                       cxpb=0.7,
                       mutpb=0.2,
                       parsimony_coefficient=0.01,
                       random_seed=42):
    """
    Ejecuta GP para sintetizar el segmento dado (0..6).
    Retorna el mejor individuo, su fitness (raw correct_count) y la expresion compilada.
    """
    random.seed(random_seed + segment_idx)  # distinta semilla por segmento

    # Construir conjunto de muestras (solo 0..9)
    samples = [int_to_bits4(n) for n in range(10)]
    target_values = [SEVEN_SEG[n][segment_idx] for n in range(10)]

    pop = toolbox.population(n=population_size)

    # Registrar evaluación en el toolbox con closure que capture target y samples
    def eval_closure(individual):
        func = toolbox.compile(expr=individual)
        # compute fitness
        return evaluate_individual(func, target_values, samples, individual, parsimony_coefficient)

    toolbox.register("evaluate", eval_closure)

    # Statistics
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", lambda x: sum(v[0] for v in x) / len(x) if x else 0.0)
    mstats.register("max", lambda x: max(v[0] for v in x) if x else 0.0)
    mstats.register("min", lambda x: min(v[0] for v in x) if x else 0.0)

    # Hall of Fame to keep best individual
    hof = tools.HallOfFame(1)

    # Evaluate initial population
    invalid = [ind for ind in pop if not ind.fitness.valid]
    for ind in invalid:
        ind.fitness.values = toolbox.evaluate(ind)

    # Evolutionary loop
    for gen in range(1, ngen + 1):
        offspring = tools.selTournament(pop, len(pop), tournsize=3)
        offspring = list(map(toolbox.clone, offspring))

        # Crossover and mutation
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < cxpb:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values
        for mutant in offspring:
            if random.random() < mutpb:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate invalid individuals
        invalid = [ind for ind in offspring if not ind.fitness.valid]
        for ind in invalid:
            ind.fitness.values = toolbox.evaluate(ind)

        pop[:] = offspring

        # Update HOF
        hof.update(pop)

    # Después de la corrida, extraer el mejor
    best = hof[0]
    # Re-evaluar to get correct_count (since fitness had parsimony)
    func_best = toolbox.compile(expr=best)
    correct = 0
    for bits, target in zip(samples, target_values):
        outb = int(bool(func_best(*bits)))
        if outb == target:
            correct += 1

    return best, correct, func_best

# -------------  Entrypoint principal -------------
def main():
    print("GP para sintetizar codificador BCD -> 7-segmentos (un GP por segmento)")
    pset = make_pset()
    toolbox = setup_deap(pset, parsimony_coefficient=0.01)

    # Parámetros (ajustables)
    POP = 300
    NGEN = 50
    CX_PB = 0.7
    MUT_PB = 0.2
    PARSIMONY = 0.01

    input_names = ['x3', 'x2', 'x1', 'x0']

    results = {}
    for seg_idx, seg_name in enumerate(['a','b','c','d','e','f','g']):
        print("\nEvolucionando segmento", seg_name, "(índice", seg_idx, ") ...")
        best, correct, func = run_gp_for_segment(
            segment_idx=seg_idx,
            pset=pset,
            toolbox=toolbox,
            population_size=POP,
            ngen=NGEN,
            cxpb=CX_PB,
            mutpb=MUT_PB,
            parsimony_coefficient=PARSIMONY,
            random_seed=1234
        )
        expr_str = str(best)
        print(f"Segmento {seg_name}: aciertos {correct}/10")
        print("  Expresión (árbol):", expr_str)
        if HAS_SYMPY:
            sym = expr_to_sympy(expr_str, input_names)
            if sym is not None:
                try:
                    simp = sp.simplify_logic(sym, form='dnf')  # o 'cnf'
                    print("  Expresión (sympy simplificada DNF):", simp)
                except Exception as e:
                    print("  Sympy fallo al simplificar:", e)
        results[seg_name] = (expr_str, correct)

    # Resumen final
    print("\n=== RESUMEN FINAL ===")
    for seg in ['a','b','c','d','e','f','g']:
        expr_str, correct = results[seg]
        print(f"{seg}: {correct}/10  --  {expr_str}")

if __name__ == "__main__":
    main()

GP para sintetizar codificador BCD -> 7-segmentos (un GP por segmento)

Evolucionando segmento a (índice 0 ) ...
Segmento a: aciertos 10/10
  Expresión (árbol): OR(XOR(x1, x3), NOT(XOR(x0, x2)))
  Expresión (sympy simplificada DNF): (x0 & x2) | (x1 & ~x3) | (x3 & ~x1) | (~x0 & ~x2)

Evolucionando segmento b (índice 1 ) ...
Segmento b: aciertos 10/10
  Expresión (árbol): OR(NOT(x2), XOR(NOT(x1), x0))
  Expresión (sympy simplificada DNF): ~x2 | (x0 & x1) | (~x0 & ~x1)

Evolucionando segmento c (índice 2 ) ...
Segmento c: aciertos 9/10
  Expresión (árbol): 1
  Expresión (sympy simplificada DNF): 1

Evolucionando segmento d (índice 3 ) ...
Segmento d: aciertos 9/10
  Expresión (árbol): XOR(XOR(x3, x1), AND(x2, x0))
  Expresión (sympy simplificada DNF): (x0 & x1 & x2 & x3) | (x1 & ~x0 & ~x3) | (x1 & ~x2 & ~x3) | (x3 & ~x0 & ~x1) | (x3 & ~x1 & ~x2) | (x0 & x2 & ~x1 & ~x3)

Evolucionando segmento e (índice 4 ) ...
Segmento e: aciertos 9/10
  Expresión (árbol): NOT(x0)
  Expresión (sympy simpl